In [65]:
import pandas as pd
import math
import sys
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import OneHotEncoder

In [66]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [67]:
DEFAULT_PATH = "../input/"
musiques = pd.read_csv(DEFAULT_PATH + "music-genre-classification/train.csv")
musiques

In [68]:
#Supprimer les données use less ou qui ne pourrait etre généraliser envers d'autre dataset
musiques_class = musiques.drop(musiques.columns[:3], axis=1)
#Création des y data
y_data = musiques_class["Class"]
musiques = musiques_class.drop(["Class"], axis=1)
musiques

In [69]:
#Existe des données null
sns.heatmap(musiques.isna())

In [70]:
#On fill les données
musiques["instrumentalness"].fillna(musiques["instrumentalness"].mean(), inplace=True)
musiques["key"].fillna(musiques["key"].mean(), inplace=True)

In [71]:
#Normalisation des données
normalized_df=(musiques-musiques.mean())/musiques.std()
normalized_df

In [72]:
#On regardes les classes et leurs distributions
plt.hist(y_data, bins=range(0,11), density=True, histtype="barstacked")

### Egalisation des données
Notament du au fait qu'elle soit injustement répartie

In [73]:
minData = musiques.shape[0]
classType = []
#Par cour les 10 class pour connaitre la class avec le moins d'éléments
for i in range(11):
    classType.append(musiques_class[musiques_class["Class"] == i])
    minSize = classType[i].shape[0]
    if minData > minSize:
        minData = minSize

musiqueClassEqual = pd.DataFrame(columns=classType[0].columns)
for df in classType:
    print(df[:minData].shape[0])
    musiqueClassEqual = pd.concat([musiqueClassEqual, df[:minData]], ignore_index=True)
    
musiqueClassEqual = musiqueClassEqual.sample(frac=1).reset_index(drop=True)
musiqueClassEqual["instrumentalness"].fillna(musiqueClassEqual["instrumentalness"].mean(), inplace=True)
musiqueClassEqual["key"].fillna(musiqueClassEqual["key"].mean(), inplace=True)
musiqueClassEqual

In [74]:
"""from collections import Counter
dict(Counter(y_data))"""

In [75]:
'''PredictX_testclassification Toujour mieux linar ==> normaliser | arbre de décistion ==> non
#Image TSNE (normalise ou virer non important) mettre que des choses pertinante
#Exploration de donner ==> google | TSNE ==> google projector
https://projector.tensorflow.org/
    https://pair-code.github.io/facets/'''

### Analyse des données
Et recherche d'un model de machine learning fonctionnel

In [76]:
plt.figure(1,figsize=(25,15)) 
sns.heatmap(musiques_class.corr(), annot=True)

In [77]:
#Linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X_train, X_test, Y_train, Y_test = train_test_split(normalized_df, y_data, test_size=.3)
#0.10
modelLr = LinearRegression()
modelLr.fit(X_train, Y_train)
print(modelLr.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()

In [78]:
"""from sklearn.cluster import KMeans

'''scores = []
for i in range(1, 50):
    model = KMeans(n_clusters = i).fit(X_train.to_numpy(), y=y_data)
    scores.append(model.inertia_)
plt.plot(range(1, 50), scores)'''"""

In [79]:
"""from sklearn.svm import SVR

#0.12
modelLr = SVR()
modelLr.fit(X_train, Y_train)
print(modelLr.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()"""

In [80]:
"""from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

#0.44
modelLr = BaggingClassifier(KNeighborsClassifier(n_neighbors=10), max_samples=0.5, max_features=0.5)
modelLr.fit(X_train, Y_train)
print(modelLr.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()"""

In [81]:
"""from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier

#0.49
model = OneVsOneClassifier(LogisticRegression())
model.fit(X_train, Y_train)

print(model.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()"""

In [82]:
"""from sklearn import svm
#0.52
modelLr = svm.SVC()
modelLr.fit(X_train, Y_train)
print(modelLr.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()"""

In [83]:
"""from sklearn.ensemble import RandomForestClassifier
#0.44
modelLr = RandomForestClassifier(n_estimators=100)
modelLr.fit(X_train, Y_train)
print(modelLr.score(X_test, Y_test))
cross_val_score(modelLr, normalized_df, y_data, cv=5).mean()"""

In [84]:
#inprouve SVM
'''modelLr = svm.SVC()
scores = []
for i in range (1,5):
    modelLr.C = i
    modelLr.fit(X_train, Y_train)
    scores.append(cross_val_score(modelLr, normalized_df, y_data, cv=5).mean())
scores'''

In [85]:
"""!pip install pycaret
from pycaret.classification import *
data = pd.read_csv("../input/music-genre-classification/train.csv")
classifier = setup(data = data, target = 'Class',ignore_features = ['Artist Name','Track Name'] ,session_id=123) """

In [86]:
"""modelLr = svm.SVC()
modelLr.fit(X_train, Y_train)
modelLr.score(X_test, Y_test)"""

In [87]:
"""predictValues = modelLr.predict(X_test)

#Detecter / normal
plt.xlabel = "Classe prédite"
plt.ylabel = "ClassNormal"
sns.heatmap(pd.crosstab(Y_test, predictValues), annot=True, fmt='d')"""

### Essaye avec un arbre de désition
#### Interpretation :
On peut constater que le model possedes des problemes pour determiner la class 1. Sur l'ensemble des données aucun a étais retenue.

Il capte également la class 2, 6, 8

Conclure :
* Les données fournie dans le dataset sont innégale (il y a plus de 10 que d'autre) Il auras donc t'endance a répondre 10 car il a plus de chance de réussir

In [88]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_leaf_nodes=10)
clf = clf.fit(X_train, Y_train)

In [89]:
clf.score(X_test, Y_test)

In [90]:
imageTree = tree.plot_tree(clf)

In [91]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("retrouve")

In [92]:
dot_data = tree.export_graphviz(clf, out_file=None, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

In [93]:
predictValues = clf.predict(X_test)

#Detecter / normal
sns.heatmap(pd.crosstab(Y_test, predictValues), annot=True, fmt='d')
plt.xlabel = "Classe prédite"
plt.ylabel = "ClassNormal"

### Importation des données qui seront entrainer

In [94]:
dfMusiquePredictGenreMusique = pd.read_csv("../input/musiquecompile/dataCompile.csv")

df = dfMusiquePredictGenreMusique.drop(dfMusiquePredictGenreMusique.columns[:5], axis=1)
df.drop(df.columns[11:16], axis=1, inplace=True)

df

In [95]:
normalized_to_predict=(df-df.mean())/df.std()
normalized_to_predict

In [96]:
classDetermine = modelLr.predict(normalized_to_predict)

In [97]:
normalized_to_predict["typeOfMusique"] = classDetermine
normalized_to_predict

In [98]:
plt.hist(normalized_to_predict["typeOfMusique"], bins=range(0,11), density=True, histtype="barstacked")

In [99]:
countDataType = normalized_to_predict["typeOfMusique"].value_counts()
cdt = countDataType
pcdt = countDataType / normalized_to_predict.shape[0] *100
lb = pd.read_csv("../input/music-genre-classification/submission.csv").columns
"""for i in range(11):
    print("{}, {}% : {}".format(cdt[i], pcdt[i], lb[i]))"""
#print(["{},{}% : {}".format(cdt[i], pcdt[i], lb[i]) for i in range(10)])
#print(countDataType, countDataType / normalized_to_predict.shape[0] *100, pd.read_csv("../input/music-genre-classification/submission.csv").columns)

In [100]:
g = sns.FacetGrid(musiques, height=4)
g.map_dataframe(sns.jointplot, "energy", "danceability", kind="hist")
g.map_dataframe(sns.jointplot, "energy", "danceability", kind="kde")
g.map_dataframe(sns.jointplot, "energy", "loudness", kind="hist")

In [101]:
g = sns.FacetGrid(normalized_to_predict, height=4)
g.map_dataframe(sns.jointplot, "energy", "typeOfMusique", kind="hist")
g.map_dataframe(sns.jointplot, "loudness", "typeOfMusique", kind="hist")

### Machine learning data egal

In [102]:
musiqueClassEqual.drop("duration_in min/ms", axis=1, inplace=True)

In [103]:
musiqueClassEqual

### Normalisation
normalized_df_Equal=(musiqueClassEqual-musiqueClassEqual.mean())/musiqueClassEqual.std()
normalized_df_Equal

In [104]:
X_train, X_test, Y_train, Y_test = train_test_split(musiqueClassEqual.drop("Class", axis=1), musiqueClassEqual["Class"], test_size=.3)

In [105]:
Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [106]:
"""clf = tree.DecisionTreeClassifier(max_leaf_nodes=10, min_samples_leaf=10)
clf = clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)
cross_val_score(clf, X_test, Y_test, cv=5).mean()"""

In [107]:
"""dot_data = tree.export_graphviz(clf, out_file=None, feature_names=musiqueClassEqual.columns[:-1], class_names = lb, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)  
graph"""

In [108]:
"""from sklearn.model_selection import GridSearchCV
dtc = tree.DecisionTreeClassifier()
print(dtc.get_params().keys())
parameters = {'max_leaf_nodes':list(range(10,250,10)), 'min_samples_leaf': list(range(10,250,10))}
clf = GridSearchCV(dtc, parameters)
clf = clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)
cross_val_score(clf, X_test, Y_test, cv=5).mean()"""

In [109]:
plt.hist(Y_test)

In [110]:
"""predictValues = dtc.predict(X_test)

#Detecter / normal
plt.xlabel = "Classe prédite"
plt.ylabel = "ClassNormal"
sns.heatmap(pd.crosstab(Y_test, predictValues), annot=True, fmt='d')"""

In [111]:
sns.clustermap(data=musiqueClassEqual.corr())

In [112]:
#Clustering

In [113]:
sns.kdeplot(data=musiqueClassEqual, x='energy', hue="Class", palette="tab10")

In [114]:
df = X_test
df["class"] = Y_test
df

In [115]:
dtc = tree.DecisionTreeClassifier(max_leaf_nodes=40, min_samples_leaf=10)
dtc.fit(X_train, Y_train)
dot_data = tree.export_graphviz(dtc, out_file=None, feature_names=musiqueClassEqual.columns[:-1], class_names = lb, filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data)
graph.render("/kaggle/output/decision_tree_graphivz.png")
graph

In [116]:
cross_val_score(dtc, X_test, Y_test, cv=15).mean()

In [117]:
df

In [118]:
df = dfMusiquePredictGenreMusique.drop(dfMusiquePredictGenreMusique.columns[:5], axis=1)
df.drop(df.columns[11:16], axis=1, inplace=True)
dataPredictClass = dtc.predict(df.drop("duration_ms", axis=1))
dataPredict = df
dataPredict["class"] = dataPredictClass

In [119]:
#df.drop("duration_ms", axis=1, inplace=True)
listmusique = df.drop("duration_ms", axis=1).drop("class", axis=1)
#listmusique = listmusique.to_numpy().reshape(1, -1)
print(listmusique.shape)

pd.DataFrame(dtc.predict_proba(listmusique), columns=None)

In [120]:
dataTrain = X_train
dataTrain["class"] = Y_train

In [121]:
for i in range(len(df.columns)-1):
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharey=True)
    axes[0].set_title("Distribution réel")
    axes[1].set_title("Distribution predite")
    sns.kdeplot(ax=axes[0] ,data=dataTrain, x=dataTrain.columns[i], hue="class", palette="tab10")
    sns.kdeplot(ax=axes[1], data=dataPredict, x=dataPredict.columns[i], hue="class", palette="tab10")
    plt.show()

In [122]:
plt.hist(dataPredict["class"])

In [123]:
#Detecter / normal
sns.heatmap(pd.crosstab(Y_test, dtc.predict(X_test.drop("class", axis=1))), annot=True, fmt='d')
plt.xlabel = "Classe prédite"
plt.ylabel = "ClassNormal"

In [124]:
X_test

### Proposition de musique en fonction du groupe

In [125]:
def proposition(musiquesLike):
    #Récuperation des data musiques
    typeMusique = [1]*11
    for musiqueLike in musiquesLike:
        typeMusique[musiqueLike[12]] += 1
        
    #Traitement
    a = dtc.predict_proba(pd.DataFrame(np.delete(musiquesLike, 12, 1)))
    
    prob = [1]*11
    for i in range(len(musiquesLike)):
        for j in range(11):
            prob[j] += typeMusique[j] * a[i][j]
    arrayNumpy = np.array(prob)
    arrayNumpy = (arrayNumpy-arrayNumpy.mean())/arrayNumpy.std() + 1
    return arrayNumpy

In [126]:
# Faire pour la class des OHE 
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

# Faire des graphique avec TSNE
# https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html
def propositionDeux(musiques):
    listDataMusique = []
    dfSimilariter = df.copy()
    enc = OneHotEncoder(handle_unknown='ignore')
    dfSimilariter["class"] = dfSimilariter["class"]
    similariter = euclidean_distances(df)
    allIdMusique = list(musiques.index)
    for i in allIdMusique:
        listDataMusique.append(similariter[i])
    dataTab = np.asarray(listDataMusique)
    dataTraitementSimilariter = (dataTab-dataTab.mean())/dataTab.std()
    dataTraitementSimilariter[0.5 <= dataTraitementSimilariter] = 0
    allData = {}
    for i in range(len(allIdMusique)):
        allData[allIdMusique[i]] = i
    a = sorted(allData, reverse=True)

    #Sortie un certain nombre
    dataPandas = []
    for i in a[:100]:
        dataPandas.append(dfMusiquePredictGenreMusique.iloc[allData[i]])
    return pd.DataFrame(dataPandas, columns=dfMusiquePredictGenreMusique.columns)

In [127]:
data = propositionDeux(X_test).drop("Unnamed: 0", axis=1)
data

In [128]:
#Donne la classe de musique qui serais le plus appressier
proposition(list(X_test.to_numpy()))